In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [ ]:
import itertools
import string
import numpy as np
import ipywidgets as widgets
from IPython.display import display,display_html

import tqdm
import pdb

import pandas as pd
# from nltk.corpus import words


In [ ]:
USE_LOCAL_CSV=True
if USE_LOCAL_CSV:
    df = pd.read_csv('df_wordl_words.csv')
else:
    sheet_url ='https://docs.google.com/spreadsheets/d/1rYkOLXEaSvq5cMY1IfpwTx4RRtIv8GNg-CyoM0Ysc84/edit#gid=2066977162'
    csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
    df = pd.read_csv(csv_export_url)
nltk_words = df.word.values
#len(nltk_words)

In [ ]:
df_ugf = pd.read_csv('unigram_freq.csv')

In [ ]:
def new_condition(word,letter_status):
    good_letters = []
    for i,letter in enumerate(list(word)):
        if letter_status[i]<3:
            good_letters.append(letter)
    def inner_test(w):
        new_conditions = []
        for i,letter in enumerate(list(word)):
            if letter_status[i]==0:
                new_conditions.append(
                    letter in w 
                )                
            elif letter_status[i]==1:
                new_conditions.append(
                    letter in w[i] 
                )                
            elif letter_status[i]==2:
                new_conditions.append(
                    (letter in w) and (letter not in w[i])
                )
            else:
                new_conditions.append(
                    letter not in w
                )
        return new_conditions
    return inner_test,good_letters

class wordl():
    def __init__(self,show_progress=True):
        self.wordlist = []
        self.letter_list = []
        self.letter_status = []
        self.conditions = []
        self.show_progress = show_progress
        all_words = set(
            nltk_words
        )
        self.nltk_words =  set(
            [
                wo 
                for wo in all_words
                if (len(wo)==5) and (wo.lower()==wo)
            ]
        )
    
    def add_word(self,word,letter_status,debug_word=''):
        self.wordlist.append(word)
        self.letter_status.append(letter_status)
        condition_list,letter_list = new_condition(word,letter_status)
        self.conditions.append(condition_list)
        self.letter_list.extend(letter_list)
        self.letter_list = list(set(self.letter_list))
    
    def conds(self,w):
        cond_results = []
        for i in range(len(self.conditions)):
            cond_results.extend(self.conditions[i](w))
        return all(cond_results)
    


    def try_it(self,letter_list=None):
        words = []
        curr_lets = self.letter_list if letter_list is None else letter_list
        poss_words = [
            w for w in self.nltk_words
            if len(set(curr_lets).intersection(w))==len(curr_lets)
        ]
        for w in poss_words:
            if self.conds(w):
                words.append(w)
        return words
        
 

In [ ]:
%%html
<style>
.mytext {
    text-transform:uppercase;
    font-weight: bold;
    color: blue;
    font-size:100px;
    text-align:center;
}
</style>


In [ ]:
perc=100

results_output = widgets.Output(
)


In [ ]:
def run_it():    
    results_output.clear_output()
    table_style = {'description_width': 'auto'}
    table_layout = {'width':'auto', 'min_width':'15px',  'min_height':'28px'}

    num_tboxes = 5
    tboxes = [
        widgets.Text(
            layout=table_layout,
    #         layout={'width':'25%'},
            style={'description_width': 'auto'} #table_style
        )
        for i in range(num_tboxes)
    ]  


    letter_status = {}
    dropdowns = []
    button_group_array = []

    def create_on_letter_button_clicked(letter_number,i):
        def on_letter_button_clicked(change):
            change.icon='check'
            letter_status[letter_number] = i
            for j in [k for k in range(3) if k != i]:
                button_group_array[letter_number][j].icon='uncheck'

        return on_letter_button_clicked

    def make_button_group(letter_number):
        button_children = []
        color_index = ["#49BE25","#D5C450","#A0A0A0"]
        for i in range(3):
            # manage icon
            icon = ''
            if i == 2:
                icon =  'check'
                letter_status[letter_number] = i
            # create button and append to button_children
            button_children.append(
                widgets.Button(style={"button_color":color_index[i]},icon=icon)
            )
            # create button observable callback for this button
            bo = create_on_letter_button_clicked(
                letter_number,i
            )
            button_children[i].on_click(bo)

        return button_children

    def make_button_groups():
        but_array = []
        for i in range(5):
            bg = make_button_group(i)
            dropdowns.append(widgets.HBox(bg))
            button_group_array.append(bg)


    make_button_groups()

    wd_box = wordl()

    def on_add_word_button_clicked(change):
        print('-----------')
        word = ''.join(
            [
                tbox.value
                for tbox in tboxes
            ]
        ).lower()
        nums = [
            letter_status[i]+1
            for i in range(5)
        ]

        print(word,nums)
        wd_box.add_word(word,nums)
        reset_word_display()

    button_add_word = widgets.Button(
        description='add_word',
        layout=widgets.Layout(width='auto', height='auto')
    )
    button_add_word.on_click(on_add_word_button_clicked)


    def on_try_it_button_clicked(change):
        words = wd_box.try_it()
        filter_words(words)
    #     display(df_ugf[df_ugf.word.isin(words)].sort_values('count',ascending=False))

    button_tryit = widgets.Button(
        description='try_it',
        layout=widgets.Layout(width='auto', height='auto')
    )
    button_tryit.on_click(on_try_it_button_clicked)

    data_entry_text = widgets.Text()
    gbox0 = widgets.GridBox(
        children=[data_entry_text],
        layout=widgets.Layout(
            display='grid',
            width=f'{perc}%',
            height='auto',
            grid_template_rows='1fr',
            grid_template_columns='1fr'
        )
    )
    def gbox_observer(text_box_changed):
        if text_box_changed['name']=='value':
            if ('new' in text_box_changed) and (len(text_box_changed['new'])>0):
                for i,t in enumerate(text_box_changed['new']):
                    tboxes[i].value = t
    data_entry_text.observe(gbox_observer)

    gbox1 = widgets.GridBox(
        children=tboxes+dropdowns,
        layout=widgets.Layout(
            display='grid',
            width=f'{perc}%',
            height='auto',
            grid_template_rows='1fr 1fr',
            grid_template_columns=' '.join(['auto' for _ in range(len(tboxes))])
        )
    )

    def tb_observer(text_box_changed):
        if text_box_changed['name']=='value':
            if ('new' in text_box_changed) and (len(text_box_changed['new'])>0):
                text_box_changed['owner'].value = text_box_changed['new'][-1].upper()

    for tbox in tboxes:
        tbox.add_class('mytext')
        tbox.observe(tb_observer)

    gbox2 = widgets.HBox(
        children=[button_add_word,button_tryit],
        layout=widgets.Layout(
            display='grid',
            width=f'{perc}%',
            height='auto',
            grid_template_rows='1fr',
            grid_template_columns=' '.join(['auto' for _ in range(2)])
        )
    )

    def reset_word_display():
        for i in range(5):
            tboxes[i].value = ''
            letter_status[i] = 2
            for j in range(2):
                button_group_array[i][j].icon='uncheck'
            button_group_array[i][2].icon='check'
        data_entry_text.value=''
        


    def display_df(df, gap=50, justify='center'):
        html= f"""
        <div style="display:flex; gap:{gap}px; justify-content:{justify};">
            {df._repr_html_()}
        </div>
        """
        return html
    
    def filter_words(words):
        with results_output:
            results_output.clear_output()
            df_new = df_ugf[df_ugf.word.isin(words)].sort_values('count',ascending=False).copy()
            count_sum = df_new['count'].sum()
            df_new['probability'] = df_new['count']/count_sum
            df_new.index = range(1,len(df_new)+1)
#             display(df_new)
            hh = display_df(df_new)
            display_html(hh, raw=True)        



    gbox3 = widgets.VBox([gbox0,gbox1,gbox2],width=f'{perc}%')
    
    hbox_layout = widgets.Layout(
        display='grid',
        width=f'{perc}%',
        height='auto',
        grid_template_rows='1fr',
        grid_template_columns='1fr 1fr',
    )

    display(
        widgets.HBox(
            [
                gbox3,
                results_output
            ],
            layout=hbox_layout
        )
    )


In [ ]:
run_it()
results_output.clear_output()
# results_output